In [13]:
import pandas as pd

data_path = "../data/raw/bf_efficiency_raw_data.csv"
df = pd.read_csv(data_path)

df = pd.read_csv(data_path)
df.shape

df.info()

numeric_cols = [
    "collection_1_output",
    "collection_2_output",
    "collection_3_output",
    "collection_4_output",
    "blood_input_ml",
    "egg_output_total"
]

for col in numeric_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(float)
    )


df[numeric_cols].dtypes

date_cols = [
    "collection_1_date",
    "collection_2_date",
    "collection_3_date",
    "collection_4_date"
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")

df[date_cols].head()

df["efficiency"] = df["egg_output_total"] / df["blood_input_ml"]
df["eggs_per_hour"] = df["egg_output_total"] / df["time_between_bf_hrs"]

df[["blood_input_ml", "egg_output_total", "efficiency", "eggs_per_hour"]].head()

df.shape

df.describe()

output_path = "../data/processed/bf_efficiency_clean.csv"
df.to_csv(output_path, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   line_id              140 non-null    object
 1   trial_id             140 non-null    object
 2   colony_strength      140 non-null    object
 3   BF1                  140 non-null    object
 4   BF2                  140 non-null    object
 5   BF3                  140 non-null    object
 6   BF4                  140 non-null    object
 7   bf1_amounts_ml       140 non-null    int64 
 8   bf2_amounts_ml       140 non-null    int64 
 9   bf3_amounts_ml       140 non-null    int64 
 10  bf4_amounts_ml       140 non-null    int64 
 11  collection_1_date    140 non-null    object
 12  collection_2_date    140 non-null    object
 13  collection_3_date    140 non-null    object
 14  collection_4_date    140 non-null    object
 15  collection_1_output  140 non-null    object
 16  collecti

In [14]:
import pandas as pd
import numpy as np

In [15]:
DATA_PATH = "../data/raw/bf_efficiency_raw_data.csv"
OUTPUT_PATH = "../data/processed/bf_efficiency_clean.csv"

df = pd.read_csv(DATA_PATH)

assert not df.empty, "Raw dataset is empty"


In [16]:
df.shape

(140, 25)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   line_id              140 non-null    object
 1   trial_id             140 non-null    object
 2   colony_strength      140 non-null    object
 3   BF1                  140 non-null    object
 4   BF2                  140 non-null    object
 5   BF3                  140 non-null    object
 6   BF4                  140 non-null    object
 7   bf1_amounts_ml       140 non-null    int64 
 8   bf2_amounts_ml       140 non-null    int64 
 9   bf3_amounts_ml       140 non-null    int64 
 10  bf4_amounts_ml       140 non-null    int64 
 11  collection_1_date    140 non-null    object
 12  collection_2_date    140 non-null    object
 13  collection_3_date    140 non-null    object
 14  collection_4_date    140 non-null    object
 15  collection_1_output  140 non-null    object
 16  collecti

In [18]:
numeric_cols = [
    "collection_1_output",
    "collection_2_output",
    "collection_3_output",
    "collection_4_output",
    "blood_input_ml",
    "egg_output_total",
    "time_between_bf_hrs"
]


In [19]:
def clean_numeric(series: pd.Series) -> pd.Series:
    cleaned = (
        series
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.strip()
    )
    return pd.to_numeric(cleaned, errors="coerce")

for col in numeric_cols:
    df[col] = clean_numeric(df[col])

In [20]:
df[numeric_cols].isna().mean().sort_values(ascending=False)


collection_1_output    0.0
collection_2_output    0.0
collection_3_output    0.0
collection_4_output    0.0
blood_input_ml         0.0
egg_output_total       0.0
time_between_bf_hrs    0.0
dtype: float64

In [21]:
date_cols = [
    "collection_1_date",
    "collection_2_date",
    "collection_3_date",
    "collection_4_date"
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")


In [22]:
df[date_cols].isna().sum()


collection_1_date    0
collection_2_date    0
collection_3_date    0
collection_4_date    0
dtype: int64

In [23]:
df[date_cols].agg(["min", "max"])


,collection_1_date,collection_2_date,collection_3_date,collection_4_date
min,2025-09-03,2025-09-05,2025-09-09,2025-09-12
max,2025-11-19,2025-11-21,2025-11-25,2025-11-28


In [24]:
(df["blood_input_ml"] < 0).sum()


np.int64(0)

In [25]:
(df["egg_output_total"] < 0).sum()


np.int64(0)

In [26]:
(df["time_between_bf_hrs"] <= 0).sum()


np.int64(0)

In [27]:
df["efficiency"] = (
    df["egg_output_total"] /
    df["blood_input_ml"].replace(0, np.nan)
)

df["eggs_per_hour"] = (
    df["egg_output_total"] /
    df["time_between_bf_hrs"].replace(0, np.nan)
)


In [28]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)


In [29]:
df[[
    "blood_input_ml",
    "egg_output_total",
    "time_between_bf_hrs",
    "efficiency",
    "eggs_per_hour"
]].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95])


,blood_input_ml,egg_output_total,time_between_bf_hrs,efficiency,eggs_per_hour
count,140.000000,1.400000e+02,140.0,140.000000,140.000000
mean,2397.714286,1.741887e+06,48.0,719.449977,36289.315476
std,413.742948,6.763886e+05,0.0,272.208005,14091.428440
min,240.000000,9.700000e+03,48.0,40.416667,202.083333
5%,2080.000000,7.338000e+05,48.0,337.875000,15287.500000
25%,2400.000000,1.317000e+06,48.0,549.375000,27437.500000
50%,2400.000000,1.754000e+06,48.0,696.250000,36541.666667
75%,2400.000000,2.081250e+06,48.0,860.416667,43359.375000
95%,3200.000000,2.861000e+06,48.0,1192.083333,59604.166667
max,3200.000000,4.355000e+06,48.0,1814.583333,90729.166667


In [30]:
df.shape

(140, 27)

In [31]:
df.head()

,line_id,trial_id,colony_strength,BF1,BF2,BF3,BF4,bf1_amounts_ml,bf2_amounts_ml,bf3_amounts_ml,...,collection_3_output,collection_4_output,time_between_bf_hrs,blood_input_ml,egg_output_total,location,humidity_pct,temperature_c,efficiency,eggs_per_hour
0,LAO BRO 20.2.1,20.2.1,"80,000",01-09-2025\n02-09-2025,04-09-2025\n05-09-2025,08-09-2025\n09-09-2025,11-09-2025\n12-09-2025,600,600,600,...,202000,218000,48,2400,1065500,22C,75,27,443.958333,22197.916667
1,LAO BRO 20.2.2,20.2.2,"80,000",01-09-2025\n02-09-2025,04-09-2025\n05-09-2025,08-09-2025\n09-09-2025,11-09-2025\n12-09-2025,600,600,600,...,187000,393000,48,2400,1155000,22C,75,27,481.250000,24062.500000
2,LAO BRO 20.2.3,20.2.3,"120,000",01-09-2025\n02-09-2025,04-09-2025\n05-09-2025,08-09-2025\n09-09-2025,11-09-2025\n12-09-2025,600,600,600,...,367000,350000,48,2400,1917000,22A,75,27,798.750000,39937.500000
3,LAO BRO 20.2.4,20.2.4,"120,000",01-09-2025\n02-09-2025,04-09-2025\n05-09-2025,08-09-2025\n09-09-2025,11-09-2025\n12-09-2025,600,600,600,...,295000,213000,48,2400,1136000,18B,75,27,473.333333,23666.666667
4,LAO BRO 20.2.5,20.2.5,"120,000",01-09-2025\n02-09-2025,04-09-2025\n05-09-2025,08-09-2025\n09-09-2025,11-09-2025\n12-09-2025,600,600,600,...,390000,200000,48,2400,1750000,18B,75,27,729.166667,36458.333333


In [32]:
df.sort_index(inplace=True)

df.to_csv(OUTPUT_PATH, index=False)

print(f"Cleaned dataset saved to: {OUTPUT_PATH}")


Cleaned dataset saved to: ../data/processed/bf_efficiency_clean.csv
